# **Импорт необходимых библиотек**

In [38]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from tapi_yandex_metrika import YandexMetrikaLogsapi
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [39]:
# импорт библиотеки ИИ
import sketch

#примеры использования
#df.sketch.ask("Which columns are integer type?")
#df.sketch.howto("Plot the sales versus time")


In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

## Ссылка на спецификации запросов Метрики

https://yandex.ru/dev/metrika/doc/api2/logs/intro.html

https://github.com/pavelmaksimov/tapi-yandex-metrika/blob/master/docs/logsapi.md

## Подготовка запроса

In [41]:
# токен доступа к метрике
ACCESS_TOKEN = "!!!!USE_YOUR_TOKEN!!!"

# номер счетчика
count_number = "61617364"

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': count_number})

In [42]:
# ВЫВОД ТЕКУЩИХ ЛОГОВ
result = client.allinfo().get()
print(result)

# ЦИКЛ ОЧИСТКИ ТЕКУЩИХ ЗАПРОСОВ
if len(result["requests"]) != 0:
    for i in range(0, len(result["requests"])):  # для того количества логов, которые есть в метритке
        req_id = result["requests"][i]["request_id"]   # получим номер очередного запроса
        res = client.clean(requestId=req_id).post()    # удалим этот запрос
        print(res)# распечатаем результат
else: print ("Логов на сервере нет")

<TapiClient object
{'requests': []}>
Логов на сервере нет


In [80]:
# запишем в переменную требуемые параметры
# дата, дата и время визита, ID визита, просмотры, новый ли пользователь, страницы входа и выходаб, глубина,
# длительность, достигнутые цели, 
# источник трафика, рекламная система, кампания Директа, группа объявлений, объявление, 
# тип платформы, платформа, тип устройства 
#  1 — десктоп, 2 — мобильные телефоны, 3 — планшеты, 4 — TV
params = {
    "fields": "ym:s:date,ym:s:dateTime,ym:s:visitID,ym:s:clientID,ym:s:watchIDs,ym:s:isNewUser,ym:s:startURL,\
ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,\
ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,\
ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory",
    "source": "visits",
    "date1": "2022-12-01",
    "date2": "2022-12-31"
}

In [81]:
# Check the possibility of creating a report. Via HTTP GET method.
result = client.evaluate().get(params=params)
print(result)

<TapiClient object
{   'log_request_evaluation': {   'expected_size': 1005580,
                                  'log_request_sum_max_size': 10737418240,
                                  'log_request_sum_size': 10736547320,
                                  'max_possible_day_quantity': 330986,
                                  'possible': True}}>


## Запрос

In [82]:
# Automatically download the report when it is prepared

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': count_number},
    # Download the report when it will be created
    wait_report=True
)

info = client.create().post(params=params)
request_id = info["log_request"]["request_id"]

report = client.download(requestId=request_id).get()

data = report.data


print("Column names")
print(report.columns)


Column names
['ym:s:date', 'ym:s:visitID', 'ym:s:clientID', 'ym:s:watchIDs', 'ym:s:isNewUser', 'ym:s:startURL', 'ym:s:endURL', 'ym:s:pageViews', 'ym:s:visitDuration', 'ym:s:goalsID', 'ym:s:lastTrafficSource', 'ym:s:lastAdvEngine', 'ym:s:lastDirectClickOrderName', 'ym:s:lastClickBannerGroupName', 'ym:s:lastDirectClickBannerName', 'ym:s:lastDirectPlatformType', 'ym:s:lastDirectPlatform', 'ym:s:deviceCategory']


## Преобразование в датафрейм

In [83]:
# Преобразование ответа Метрики в датафрейм   ()      
file = open("cashe.csv", "w", encoding="utf-8")   # сздаем и открываем новый файл
file.write(data)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=0, sep='\t', index_col=0, encoding='utf-8')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

,ym:s:date,ym:s:visitID,ym:s:clientID,ym:s:watchIDs,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory
0,2022-12-02,7166621577389277251,1669975291682516483,[7166621577389277251],1,https://kassa.life/arenda,https://kassa.life/arenda,1,0,[],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса с эквайрингом в аренду за 1400 ₽/мес. Все включено,Context,dsp-opera-browser.yandex.ru,2
1,2022-12-02,7161123543182737712,1669954321837598400,"[7161123543182737712,7161127472546971891]",1,https://kassa.life/arenda,https://kassa.life/arenda,1,15,[],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,com.lyxoto.master.forminecraftpe,2
2,2022-12-21,7594437464571772991,1671607278486843246,[7594437464571772991],1,"https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145","https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145",1,0,[],ad,ya_undefined,0,NaN,72057607253447980,NaN,NaN,2
3,2022-12-21,7594789944198955108,1671608623254784067,"[7594789944198955108,7594791986149458225,7594791988028768356,7594810191983673613]",1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/tilda/click/rec487399334/button2,2,77,[202896550],ad,ya_direct,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,com.EmZi.BubbleKlass,2
4,2022-12-18,7534222670115373374,1671377576702224136,[7534222670115373374],1,https://kassa.life/evotor5,https://kassa.life/evotor5,1,0,[],ad,ya_direct,Кассы / Эвотор 5 / МК,Кассы / Эвотор 5 / МК,"Онлайн-касса Эвотор 5 за 9900 руб! Б/у, «как новая»",Context,com.europosit.pixelcoloring,2


In [117]:
# посчитаем количество достигнутых целей по каждому визиту
# для этого возьмем длину массива в каждой ячейке столбца достижения целей
# вычтем 2 (скобки) и разделим на 9 - длина номера цели
df['goals_achieved'] = round((df['ym:s:goalsID'].apply(lambda x: len(x))-2)/9, 0)

In [115]:
df.head()

,ym:s:date,ym:s:visitID,ym:s:clientID,ym:s:watchIDs,ym:s:isNewUser,ym:s:startURL,ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:goalsID,ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastDirectClickOrderName,ym:s:lastClickBannerGroupName,ym:s:lastDirectClickBannerName,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform,ym:s:deviceCategory,goals_achieved
0,2022-12-02,7166621577389277251,1669975291682516483,[7166621577389277251],1,https://kassa.life/arenda,https://kassa.life/arenda,1,0,[],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса с эквайрингом в аренду за 1400 ₽/мес. Все включено,Context,dsp-opera-browser.yandex.ru,2,0.0
1,2022-12-02,7161123543182737712,1669954321837598400,"[7161123543182737712,7161127472546971891]",1,https://kassa.life/arenda,https://kassa.life/arenda,1,15,[],ad,ya_direct,Аренда МК / Ш / 700р — 2,Аренда МК / Ш / 700р — 2,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,Context,com.lyxoto.master.forminecraftpe,2,0.0
2,2022-12-21,7594437464571772991,1671607278486843246,[7594437464571772991],1,"https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145","https://kassa.life/zamena-fn-vladimir/?stat-id=43&test-tag=460145616280081&banner-sizes=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjM0MHg1MTgifQ%3D%3D&format-type=118&actual-format=14&pcodever=697157&banner-test-tags=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6IjU4MTY0OSJ9&order-banners-options=eyI3MjA1NzYwNzI1MzQ0Nzk4MSI6MjA0OH0&pcode-active-testids=694688,0,64;695807,0,38&width=340&height=518&clickX=260&clickY=145",1,0,[],ad,ya_undefined,0,NaN,72057607253447980,NaN,NaN,2,0.0
3,2022-12-21,7594789944198955108,1671608623254784067,"[7594789944198955108,7594791986149458225,7594791988028768356,7594810191983673613]",1,https://kassa.life/zamena-fn-vladimir/,https://kassa.life/tilda/click/rec487399334/button2,2,77,[202896550],ad,ya_direct,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,"Фискальный накопитель для Атол, Эвотор и др. От 9400 руб",Context,com.EmZi.BubbleKlass,2,1.0
4,2022-12-18,7534222670115373374,1671377576702224136,[7534222670115373374],1,https://kassa.life/evotor5,https://kassa.life/evotor5,1,0,[],ad,ya_direct,Кассы / Эвотор 5 / МК,Кассы / Эвотор 5 / МК,"Онлайн-касса Эвотор 5 за 9900 руб! Б/у, «как новая»",Context,com.europosit.pixelcoloring,2,0.0


In [137]:
# ВЫВОД ТЕКУЩИХ ЛОГОВ
result = client.allinfo().get()
print(result)

# ЦИКЛ ОЧИСТКИ ТЕКУЩИХ ЗАПРОСОВ
if len(result["requests"]) != 0:
    for i in range(0, len(result["requests"])):  # для того количества логов, которые есть в метритке
        req_id = result["requests"][i]["request_id"]   # получим номер очередного запроса
        res = client.clean(requestId=req_id).post()    # удалим этот запрос
        print(res)# распечатаем результат
else: print ("Логов на сервере нет")

<TapiClient object
{   'requests': [   {   'attribution': 'LASTSIGN',
                        'counter_id': 61617364,
                        'date1': '2022-12-01',
                        'date2': '2022-12-31',
                        'fields': [   'ym:s:date',
                                      'ym:s:visitID',
                                      'ym:s:watchIDs',
                                      'ym:s:isNewUser',
                                      'ym:s:startURL',
                                      'ym:s:endURL',
                                      'ym:s:pageViews',
                                      'ym:s:visitDuration',
                                      'ym:s:goalsID',
                                      'ym:s:lastTrafficSource',
                                      'ym:s:lastAdvEngine',
                                      'ym:s:lastDirectClickOrderName',
                                      'ym:s:lastClickBannerGroupName',
                            

## *Функция группировки данных по одному параметру (group_df)*

In [136]:
def group_df(df, column):
    # сгруппируем 
    #
    grouped_device = df.groupby(column).agg({'ym:s:visitID': ['count'], 'ym:s:clientID': ['count', 'nunique'], 
                                             'ym:s:isNewUser' : ['mean'], 'ym:s:pageViews' : ['mean'], 
                                             'ym:s:visitDuration' : ['mean'], 'goals_achieved' : ['sum','mean']}).reset_index()
    grouped_device.columns = [column, 'Визитов', 'Клиентов', 'Уник кл-в', 'Доля новых кл-в', 
                              'Просм страниц', 'Ср длит визита', 'Достигнуто целей' , 'Конверсия визит в цель']
    
    #grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('Доля новый кл-в', 'Просм страниц', 'Ср длит визита', 'Конверсия визит в цель'):
        grouped_device[col] = round((grouped_device[col]), 2)
    
    return(grouped_device)

In [133]:
df_df = group_df(df, 'ym:s:date')
df_df

,ym:s:date,Визитов,Клиентов,Уник кл-в,Доля новый кл-в,Просм страниц,Ср длит визита,Достигнуто целей,Конверсия визит в цель
0,2022-12-01,53,53,48,0.70,2.62,195.13,14.0,0.26
1,2022-12-02,393,393,353,0.89,1.12,36.18,13.0,0.03
2,2022-12-03,32,32,28,0.38,1.41,16.78,1.0,0.03
3,2022-12-04,15,15,11,0.40,2.73,126.07,8.0,0.53
4,2022-12-05,65,65,55,0.68,1.51,52.00,9.0,0.14
5,2022-12-06,34,34,30,0.62,2.26,98.56,9.0,0.26
6,2022-12-07,40,40,32,0.60,1.75,80.12,9.0,0.22
7,2022-12-08,118,118,94,0.65,1.74,74.86,25.0,0.21
8,2022-12-09,130,130,115,0.81,1.40,77.28,30.0,0.23
9,2022-12-10,28,28,20,0.43,2.18,119.64,7.0,0.25


## *Функция группировки по двум пар-м (group_df_2_param)*

In [134]:
def group_df_2_param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby([column1, column2] ).agg({'ym:s:visitID': ['count'], 'ym:s:clientID': ['count', 'nunique'], 
                                             'ym:s:isNewUser' : ['mean'], 'ym:s:pageViews' : ['mean'], 
                                             'ym:s:visitDuration' : ['mean'], 'goals_achieved' : ['sum','mean']}).reset_index()
    grouped_device.columns = [column1, column2, 'Визитов', 'Клиентов', 'Уник кл-в', 'Доля новый кл-в', 
                              'Просм страниц', 'Ср длит визита', 'Достигнуто целей' , 'Конверсия визит в цель']

    for col in ('Доля новый кл-в', 'Просм страниц', 'Ср длит визита', 'Конверсия визит в цель'):
        grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_device)

In [135]:
df_df = group_df_2_param(df, 'ym:s:date', 'ym:s:isNewUser')
df_df

,ym:s:date,ym:s:isNewUser,Визитов,Клиентов,Уник кл-в,Доля новый кл-в,Просм страниц,Ср длит визита,Достигнуто целей,Конверсия визит в цель
0,2022-12-01,0,16,16,13,0.0,1.94,214.88,3.0,0.19
1,2022-12-01,1,37,37,37,1.0,2.92,186.59,11.0,0.30
2,2022-12-02,0,42,42,35,0.0,1.10,17.71,1.0,0.02
3,2022-12-02,1,351,351,349,1.0,1.13,38.39,12.0,0.03
4,2022-12-03,0,20,20,16,0.0,1.55,20.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...
57,2022-12-29,1,57,57,57,1.0,1.18,21.14,1.0,0.02
58,2022-12-30,0,7,7,7,0.0,2.57,217.57,0.0,0.00
59,2022-12-30,1,14,14,14,1.0,2.00,132.14,7.0,0.50
60,2022-12-31,0,2,2,2,0.0,1.00,16.00,0.0,0.00
